In [1]:
import numpy as np
from numpy import sin, cos, pi, floor, exp, log, log10, sqrt, cbrt
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

In [4]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:06<00:00, 3966081.27it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 99385.42it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:10<00:00, 420842.88it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 2118551.51it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [5]:
batch_size = 64

# データローダーの作成
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [8]:
# 訓練に際して、可能であればGPU（mps）を設定します。GPUが搭載されていない場合はCPUを使用します
device = torch.device('mps' if torch.backends.mps.is_available() else "cpu") 
print("Using {} device".format(device))

# modelを定義します
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [19]:
for a in vars(model):
# for a in dir(model):
#     if a in vars(model):
#         continue
#     if callable(getattr(model, a)):
#         continue
    print(a)

training
_parameters
_buffers
_non_persistent_buffers_set
_backward_pre_hooks
_backward_hooks
_is_full_backward_hook
_forward_hooks
_forward_hooks_with_kwargs
_forward_pre_hooks
_forward_pre_hooks_with_kwargs
_state_dict_hooks
_state_dict_pre_hooks
_load_state_dict_pre_hooks
_load_state_dict_post_hooks
_modules


In [21]:
print(next(model.parameters()).device)

mps:0


In [22]:
learning_rate = 1e-3
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [23]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # 損失誤差を計算
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [24]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [28]:
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")



Epoch 1
-------------------------------
loss: 0.805927  [    0/60000]
loss: 0.813849  [ 6400/60000]
loss: 0.730569  [12800/60000]
loss: 1.077690  [19200/60000]
loss: 0.883708  [25600/60000]
loss: 1.031282  [32000/60000]
loss: 1.038801  [38400/60000]
loss: 1.213971  [44800/60000]
loss: 1.004539  [51200/60000]
loss: 0.990750  [57600/60000]
Test Error: 
 Accuracy: 66.3%, Avg loss: 0.015178 

Epoch 2
-------------------------------
loss: 0.798586  [    0/60000]
loss: 0.808363  [ 6400/60000]
loss: 0.724709  [12800/60000]
loss: 1.072125  [19200/60000]
loss: 0.877799  [25600/60000]
loss: 1.026748  [32000/60000]
loss: 1.033231  [38400/60000]
loss: 1.212337  [44800/60000]
loss: 1.001811  [51200/60000]
loss: 0.983933  [57600/60000]
Test Error: 
 Accuracy: 66.3%, Avg loss: 0.015105 

Epoch 3
-------------------------------
loss: 0.791623  [    0/60000]
loss: 0.803356  [ 6400/60000]
loss: 0.719058  [12800/60000]
loss: 1.066631  [19200/60000]
loss: 0.872718  [25600/60000]
loss: 1.022065  [32000/600

In [29]:
torch.save(model.state_dict(), "model0913.pth")
print("saved to model.pth")

saved to model.pth


In [30]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model0913.pth"))

<All keys matched successfully>

In [32]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[10][0], test_data[10][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Coat", Actual: "Coat"


In [3]:
bool('False')

True